<a href="https://colab.research.google.com/github/robgon-art/ai-tunes/blob/main/AI_Tunes_GPT_3_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AI-Tunes: Creating New Songs with Artificial Intelligence**
### **How I fine-tuned OpenAI's GPT-3 to generate music with a global structure**
By Robert. A Gonsalves

You can read my article about this project on [Medium](https://towardsdatascience.com/ai-tunes-creating-new-songs-with-artificial-intelligence-4fb383218146).

The source code is released under the CC BY-SA license.

![CC BY-NC-SA](https://licensebuttons.net/l/by-sa/4.0/88x31.png)


##**Initialize the System**

In [ ]:
!git clone https://github.com/robgon-art/music-geometry-eval
!git clone https://github.com/00sapo/OpenEWLD
!gsutil -q -m cp -r gs://magentadata/models/music_transformer/primers/* /content/
!gsutil -q -m cp gs://magentadata/soundfonts/Yamaha-C5-Salamander-JNv5.1.sf2 /content/
!apt-get update -qq && apt-get install -qq libfluidsynth1 build-essential libasound2-dev libjack-dev
!pip install magenta
!pip install pyfluidsynth
!pip install openai
#import note_seq
SF2_PATH = '/content/Yamaha-C5-Salamander-JNv5.1.sf2'
SAMPLE_RATE = 16000

!wget https://wim.vree.org/svgParse/xml2abc.py-143.zip
!unzip xml2abc.py-143.zip
import sys
sys.path.append('/content/music-geometry-eval/music_geometry_eval')
#import music_geometry_eval
import glob
import random
import music21
#import music_geometry_eval
from collections.abc import Iterable
import numpy as np

##**Evaluate the Song Files**

In [7]:
!wget https://wim.vree.org/svgParse/xml2abc.py-143.zip
!unzip xml2abc.py-143.zip

--2023-05-23 12:44:05--  https://wim.vree.org/svgParse/xml2abc.py-143.zip
Resolving wim.vree.org (wim.vree.org)... 217.160.212.185
Connecting to wim.vree.org (wim.vree.org)|217.160.212.185|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2023-05-23 12:44:05 ERROR 404: Not Found.

unzip:  cannot find or open xml2abc.py-143.zip, xml2abc.py-143.zip.zip or xml2abc.py-143.zip.ZIP.


In [2]:
import sys
import glob
import random
import music21
#import music_geometry_eval
from collections.abc import Iterable
import numpy as np
random.seed(42)
song_files = glob.glob("OpenEWLD/dataset/*/*/*.mxl")



random.seed(42)
song_files = glob.glob("OpenEWLD/dataset/*/*/*.mxl")
random.shuffle(song_files)
num_files = len(song_files)
print(song_files)
print("number of song files is", num_files)

transpose_dict = {"G major": 5, "A- major": 4, "A major": 3, "B- major": 2, "B major": 1, "C major": 0, "D- major": -1, "D major": -2, "E- major": -3, "E major": -4, "F major": -5, "F# major": -6}

['OpenEWLD/dataset/Lorenz_Hart-Richard_Rodgers/A_Ship_Without_a_Sail/A_Ship_Without_a_Sail.mxl', 'OpenEWLD/dataset/Johnny_Mercer-Victor_Schertzinger/Tangerine/Tangerine.mxl', 'OpenEWLD/dataset/Billy_Rose-Edward_Eliscu-Vincent_Youmans/Without_a_Song/Without_a_Song.mxl', 'OpenEWLD/dataset/Howard_Johnson-Theodore_Morse/M-O-T-H-E-R_(A_Word_That_Means_the_World_to_Me)/M-O-T-H-E-R_(A_Word_That_Means_the_World_to_Me).mxl', 'OpenEWLD/dataset/Al_Dubin-Harry_Warren/September_in_the_Rain/September_in_the_Rain.mxl', "OpenEWLD/dataset/Jerome_Kern-Johnny_Mercer/I'm_Old_Fashioned/I'm_Old_Fashioned.mxl", 'OpenEWLD/dataset/Anne_Caldwell-Vincent_Youmans/I_Know_That_You_Know/I_Know_That_You_Know.mxl', 'OpenEWLD/dataset/Gus_Kahn-Walter_Donaldson/I_Wonder_Where_My_Baby_Is_Tonight/I_Wonder_Where_My_Baby_Is_Tonight.mxl', 'OpenEWLD/dataset/Lorenz_Hart-Richard_Rodgers/Mountain_Greenery/Mountain_Greenery.mxl', 'OpenEWLD/dataset/Joseph_LaCalle/Amapola/Amapola.mxl', "OpenEWLD/dataset/Walter_Donaldson/You're_Drivi

In [4]:
keys = {}
metres = {}

cmm_arr = np.empty((0), np.float32)
lm_arr = np.empty((0), np.float32)
cent_arr = np.empty((0), np.float32)

for s in song_files:
  print("\n" + s)
  score = music21.converter.parse(s)

  key = None
  metre = None
  part = score.parts[0]
  for p in part:
    if isinstance(p, Iterable):
      for n in p:
        if type(n) == music21.key.Key:
          key = n.name
        if type(n) == music21.meter.TimeSignature:
          metre = n.ratioString

  if metre in metres.keys():
    metres[metre] += 1
  else:
    metres[metre] = 1

  if key in keys.keys():
    keys[key] += 1
  else:
    keys[key] = 1

  if not (metre == "4/4" or metre == "2/2"):
    continue

  if not "major" in key:
    continue

  print(key, metre)

  if key in keys.keys():
    keys[key] += 1
  else:
    keys[key] = 1

  if key in transpose_dict.keys():
    interval = transpose_dict[key]
    print("transposing from key", key, "to C major using interval", interval)
    score = score.transpose(interval)

  note_array = []

  for p in part:
    if isinstance(p, Iterable):
      for n in p:
        if type(n) == music21.note.Note:
          note_array.append([int(n.pitch.ps), int(n.quarterLength*4+0.5)])

  # print(note_array)

print(metres)
print(keys)


OpenEWLD/dataset/Lorenz_Hart-Richard_Rodgers/A_Ship_Without_a_Sail/A_Ship_Without_a_Sail.mxl
E- major 4/4
transposing from key E- major to C major using interval -3

OpenEWLD/dataset/Johnny_Mercer-Victor_Schertzinger/Tangerine/Tangerine.mxl
C major 4/4
transposing from key C major to C major using interval 0

OpenEWLD/dataset/Billy_Rose-Edward_Eliscu-Vincent_Youmans/Without_a_Song/Without_a_Song.mxl
E- major 4/4
transposing from key E- major to C major using interval -3

OpenEWLD/dataset/Howard_Johnson-Theodore_Morse/M-O-T-H-E-R_(A_Word_That_Means_the_World_to_Me)/M-O-T-H-E-R_(A_Word_That_Means_the_World_to_Me).mxl
E- major 2/2
transposing from key E- major to C major using interval -3

OpenEWLD/dataset/Al_Dubin-Harry_Warren/September_in_the_Rain/September_in_the_Rain.mxl
C major 4/4
transposing from key C major to C major using interval 0

OpenEWLD/dataset/Jerome_Kern-Johnny_Mercer/I'm_Old_Fashioned/I'm_Old_Fashioned.mxl
F major 4/4
transposing from key F major to C major using inter


OpenEWLD/dataset/John_Baptiste_Calkin/I_Heard_the_Bells_on_Christmas_Day/I_Heard_the_Bells_on_Christmas_Day.mxl
C major 4/4
transposing from key C major to C major using interval 0

OpenEWLD/dataset/Charles_C._Converse-Joseph_M._Scriven/What_a_Friend_We_Have_in_Jesus/What_a_Friend_We_Have_in_Jesus.mxl
F major 4/4
transposing from key F major to C major using interval -5

OpenEWLD/dataset/Bing_Crosby-Fred_E._Ahlert-Roy_Turk/Where_the_Blue_of_the_Night_Meets_the_Gold_of_the_Day/Where_the_Blue_of_the_Night_Meets_the_Gold_of_the_Day.mxl

OpenEWLD/dataset/Andy_Razaf-Fats_Waller/A_Hopeless_Love_Affair/A_Hopeless_Love_Affair.mxl
F major 4/4
transposing from key F major to C major using interval -5

OpenEWLD/dataset/Bertha_Scott-Jimmy_Henderson_[1]-Sid_Robin/I_Miss_You_So/I_Miss_You_So.mxl
G major 4/4
transposing from key G major to C major using interval 5

OpenEWLD/dataset/Franz_Liszt/Liebestraum_Nr._3/Liebestraum_Nr._3.mxl

OpenEWLD/dataset/Billy_Hill-Peter_De_Rose/Have_You_Ever_Been_Lonel


OpenEWLD/dataset/Con_Conrad-Sidney_Clare/Ma,_He's_Making_Eyes_at_Me/Ma,_He's_Making_Eyes_at_Me.mxl
C major 4/4
transposing from key C major to C major using interval 0

OpenEWLD/dataset/Lorenz_Hart-Richard_Rodgers/Have_You_Met_Miss_Jones?/Have_You_Met_Miss_Jones?.mxl
F major 4/4
transposing from key F major to C major using interval -5

OpenEWLD/dataset/James_Waddel_Alexander/O_Sacred_Head,_Now_Wounded/O_Sacred_Head,_Now_Wounded.mxl
C major 4/4
transposing from key C major to C major using interval 0

OpenEWLD/dataset/Jerome_Kern-Oscar_Hammerstein_II/Make_Believe/Make_Believe.mxl
C major 4/4
transposing from key C major to C major using interval 0

OpenEWLD/dataset/Jimmy_Monaco-Johnny_Burke/An_Apple_for_the_Teacher/An_Apple_for_the_Teacher.mxl
C major 4/4
transposing from key C major to C major using interval 0

OpenEWLD/dataset/Gus_Kahn-Isham_Jones/On_the_Alamo/On_the_Alamo.mxl
E- major 4/4
transposing from key E- major to C major using interval -3

OpenEWLD/dataset/Leo_Robin-Ralph_R

E- major 4/4
transposing from key E- major to C major using interval -3

OpenEWLD/dataset/Charles_Wesley-Felix_Mendelssohn/Hark!_The_Herald_Angels_Sing/Hark!_The_Herald_Angels_Sing.mxl
G major 4/4
transposing from key G major to C major using interval 5

OpenEWLD/dataset/Ira_Gershwin-Jerome_Kern/Long_Ago_and_Far_Away/Long_Ago_and_Far_Away.mxl
F major 2/2
transposing from key F major to C major using interval -5

OpenEWLD/dataset/George_Gershwin-Ira_Gershwin/Embraceable_You/Embraceable_You.mxl
G major 4/4
transposing from key G major to C major using interval 5

OpenEWLD/dataset/Jerome_Kern-Oscar_Hammerstein_II/Ol'_Man_River/Ol'_Man_River.mxl
E- major 4/4
transposing from key E- major to C major using interval -3

OpenEWLD/dataset/Jerome_Kern-Leo_Robin/In_Love_in_Vain/In_Love_in_Vain.mxl
E- major 4/4
transposing from key E- major to C major using interval -3

OpenEWLD/dataset/Charles_N._Daniels-Gus_Arnheim-Harry_Tobias/Sweet_and_Lovely/Sweet_and_Lovely.mxl
C major 4/4
transposing from k


OpenEWLD/dataset/Fred_Godfrey-Mark_Sheridan/Who_Were_You_with_Last_Night/Who_Were_You_with_Last_Night.mxl

OpenEWLD/dataset/Rida_Johnson_Young-Victor_Herbert/I'm_Falling_in_Love_with_Someone/I'm_Falling_in_Love_with_Someone.mxl

OpenEWLD/dataset/Al_Dubin-Fred_Rath-Jos_Garren/Just_a_Girl_That_Men_Forget/Just_a_Girl_That_Men_Forget.mxl

OpenEWLD/dataset/Frederick_Oakeley/O_Come,_All_Ye_Faithful/O_Come,_All_Ye_Faithful.mxl
G major 4/4
transposing from key G major to C major using interval 5

OpenEWLD/dataset/Dorothy_Parker-Ralph_Rainger/I_Wished_on_the_Moon/I_Wished_on_the_Moon.mxl
B- major 4/4
transposing from key B- major to C major using interval 2

OpenEWLD/dataset/[Traditional]/The_Quartermaster's_Stores/The_Quartermaster's_Stores.mxl
G major 4/4
transposing from key G major to C major using interval 5

OpenEWLD/dataset/Leo_Robin-Ralph_Rainger/June_in_January/June_in_January.mxl
E- major 4/4
transposing from key E- major to C major using interval -3

OpenEWLD/dataset/[Traditional]/S

G major 4/4
transposing from key G major to C major using interval 5

OpenEWLD/dataset/Annie_MacLeod-Sir_Harold_Boulton/The_Skye_Boat_Song/The_Skye_Boat_Song.mxl

OpenEWLD/dataset/B.G._DeSylva-George_Gershwin/Do_It_Again/Do_It_Again.mxl
F major 2/2
transposing from key F major to C major using interval -5

OpenEWLD/dataset/Dorothy_Fields-Jerome_Kern/A_Fine_Romance/A_Fine_Romance.mxl
C major 4/4
transposing from key C major to C major using interval 0

OpenEWLD/dataset/Frank_Loesser-George_Terry-Victor_Schertzinger/Kiss_the_Boys_Goodbye/Kiss_the_Boys_Goodbye.mxl
C major 4/4
transposing from key C major to C major using interval 0

OpenEWLD/dataset/Lorenzo_Barcelata_Castro/Maria_Elena/Maria_Elena.mxl
C major 4/4
transposing from key C major to C major using interval 0

OpenEWLD/dataset/Dick_Smith-Felix_Bernard/Winter_Wonderland/Winter_Wonderland.mxl
C major 4/4
transposing from key C major to C major using interval 0

OpenEWLD/dataset/Lorenz_Hart-Richard_Rodgers/Thou_Swell/Thou_Swell.mxl


OpenEWLD/dataset/Henry_Creamer-James_P._Johnson/If_I_Could_Be_with_You_(One_Hour_Tonight)/If_I_Could_Be_with_You_(One_Hour_Tonight).mxl
C major 2/2
transposing from key C major to C major using interval 0

OpenEWLD/dataset/C._W._Murphy-Dan_Lipton-John_Neat/She's_a_Lassie_from_Lancashire/She's_a_Lassie_from_Lancashire.mxl

OpenEWLD/dataset/George_Gershwin-Ira_Gershwin/Love_Walked_In/Love_Walked_In.mxl
E- major 2/2
transposing from key E- major to C major using interval -3

OpenEWLD/dataset/George_Gershwin-Ira_Gershwin/Soon/Soon.mxl
E- major 2/2
transposing from key E- major to C major using interval -3

OpenEWLD/dataset/Harry_Tierney-Joseph_McCarthy/Alice_Blue_Gown/Alice_Blue_Gown.mxl

OpenEWLD/dataset/Fred_E._Ahlert-Roy_Turk/Walkin'_My_Baby_Back_Home/Walkin'_My_Baby_Back_Home.mxl
E- major 4/4
transposing from key E- major to C major using interval -3

OpenEWLD/dataset/[Traditional]/Alouette/Alouette.mxl
F major 4/4
transposing from key F major to C major using interval -5

OpenEWLD/da

F major 4/4
transposing from key F major to C major using interval -5

OpenEWLD/dataset/[Traditional]/Frankie_and_Johnny/Frankie_and_Johnny.mxl
C major 4/4
transposing from key C major to C major using interval 0

OpenEWLD/dataset/Eddie_Leonard-Eddie_Munson/Ida!_Sweet_as_Apple_Cider/Ida!_Sweet_as_Apple_Cider.mxl
C major 4/4
transposing from key C major to C major using interval 0

OpenEWLD/dataset/Alfredo_Le_Pera-Carlos_Gardel/Por_una_cabeza/Por_una_cabeza.mxl

OpenEWLD/dataset/[Traditional]/Wexford_Carol/Wexford_Carol.mxl

OpenEWLD/dataset/Al_Jolson-B.G._DeSylva-Vincent_Rose/Avalon/Avalon.mxl
C major 4/4
transposing from key C major to C major using interval 0

OpenEWLD/dataset/Grant_Clarke-Harry_Akst/Am_I_Blue?/Am_I_Blue?.mxl
F major 2/2
transposing from key F major to C major using interval -5

OpenEWLD/dataset/Lew_Pollack/That's_a_Plenty/That's_a_Plenty.mxl
F major 2/2
transposing from key F major to C major using interval -5

OpenEWLD/dataset/Leo_Robin-Newell_Chase-Richard_A._Whit

E- major 4/4
transposing from key E- major to C major using interval -3

OpenEWLD/dataset/[Traditional]/The_Prisoner's_Song/The_Prisoner's_Song.mxl
D major 4/4
transposing from key D major to C major using interval -2

OpenEWLD/dataset/Harry_B._Smith/Yours_Is_My_Heart_Alone/Yours_Is_My_Heart_Alone.mxl
F major 2/2
transposing from key F major to C major using interval -5

OpenEWLD/dataset/Egbert_Van_Alstyne-Harry_Williams/In_the_Shade_of_the_Old_Apple_Tree/In_the_Shade_of_the_Old_Apple_Tree.mxl

OpenEWLD/dataset/Stephen_Foster/Old_Folks_at_Home/Old_Folks_at_Home.mxl
C major 4/4
transposing from key C major to C major using interval 0

OpenEWLD/dataset/Lorenz_Hart-Richard_Rodgers/This_Funny_World/This_Funny_World.mxl
G major 4/4
transposing from key G major to C major using interval 5

OpenEWLD/dataset/Paul_Dresser/My_Gal_Sal/My_Gal_Sal.mxl

OpenEWLD/dataset/Jerome_Kern-Oscar_Hammerstein_II/All_the_Things_You_Are/All_the_Things_You_Are.mxl
A- major 4/4
transposing from key A- major to C 

## **Prepare the Training Data**

In [6]:
import subprocess
from collections.abc import Iterable

num_prompts = 0
prompt_file = open("songs.jsonl", "w")
prompts = []
original_songs = []
for s in song_files:
  print(s)
  score = music21.converter.parse(s)

  key = None
  metre = None
  part = score.parts[0]
  for p in part:
    if isinstance(p, Iterable):
      for n in p:
        if type(n) == music21.key.Key:
          key = n.name
        if type(n) == music21.meter.TimeSignature:
          metre = n.ratioString
  print(key, metre)

  if not (metre == "4/4" or metre == "2/2"):
    continue

  if not "major" in key:
    continue

  if key in transpose_dict.keys():
    interval = transpose_dict[key]
    print("transposing from key", key, "to C major using interval", interval)
    score = score.transpose(interval)

  score.write('xml', fp='song.xml')

  try:
    output_bytes = subprocess.check_output(["python", "/content/xml2abc_143/xml2abc.py", "song.xml", "-u", "-d", "4"], timeout=5)
    output = output_bytes.decode("utf-8").strip()
    with open("song.txt", "w") as abc_file:
      abc_file.write(output)
  except:
    print("Unexpected error:", sys.exc_info()[0])
    continue

  showed_title = False
  prompt_string = ""
  completion_string = ""
  is_header = True
  with open("song.txt") as song_file:
    lines = song_file.readlines()
    for line in lines:
      line = line.replace("$", "")
      line = line.replace("dc=", "")
      line = line.strip()

      if line.startswith("V:"):
        is_header = False

      if is_header:
        if line.startswith("X:") or line.startswith("C:"):
          prompt_string += line+"\n"

        if line.startswith("T:") and not showed_title:
            prompt_string += line+"\n"
            showed_title = True
      else:
        if not line.startswith("w:") and not line.startswith("V:"):

          # remove end of line comments
          index = line.rfind('%')
          if index > 0:
            line = line[:index].strip()

          # remove inline comments
          parts = line.split('"')
          newline = ""
          for i, p in enumerate(parts):
            if i%2 == 0:
              newline += p
            elif not p.startswith("^"):
              newline += '"' + p + '"'
          line = ' '.join(newline.split())

          completion_string += line+"\n"

      if line.startswith("V:"):
        is_header = False

    prompt_string = prompt_string.replace(":",": ")
    prompt_string = prompt_string.replace('"', "`")
    prompt_string = prompt_string.replace("\n"," $ ")

    completion_string = completion_string.replace('"', "`")
    completion_string = completion_string.strip().replace("\n"," $ ")

    prompt = '{"prompt": "' + prompt_string + '", '
    prompt += '"completion": " ' + completion_string + ' $ "}\n'

    if prompt not in prompts:
      original_songs.append(s)
      prompt_file.write(prompt)
      prompts.append(prompt)
      num_prompts += 1

prompt_file.close()
print("num prompts is", num_prompts)
     

OpenEWLD/dataset/Lorenz_Hart-Richard_Rodgers/A_Ship_Without_a_Sail/A_Ship_Without_a_Sail.mxl
E- major 4/4
transposing from key E- major to C major using interval -3


python: can't open file '/content/xml2abc_143/xml2abc.py': [Errno 2] No such file or directory
python: can't open file '/content/xml2abc_143/xml2abc.py': [Errno 2] No such file or directory


Unexpected error: <class 'subprocess.CalledProcessError'>
OpenEWLD/dataset/Johnny_Mercer-Victor_Schertzinger/Tangerine/Tangerine.mxl
C major 4/4
transposing from key C major to C major using interval 0
Unexpected error: <class 'subprocess.CalledProcessError'>
OpenEWLD/dataset/Billy_Rose-Edward_Eliscu-Vincent_Youmans/Without_a_Song/Without_a_Song.mxl
E- major 4/4
transposing from key E- major to C major using interval -3


python: can't open file '/content/xml2abc_143/xml2abc.py': [Errno 2] No such file or directory
python: can't open file '/content/xml2abc_143/xml2abc.py': [Errno 2] No such file or directory


Unexpected error: <class 'subprocess.CalledProcessError'>
OpenEWLD/dataset/Howard_Johnson-Theodore_Morse/M-O-T-H-E-R_(A_Word_That_Means_the_World_to_Me)/M-O-T-H-E-R_(A_Word_That_Means_the_World_to_Me).mxl
E- major 2/2
transposing from key E- major to C major using interval -3
Unexpected error: <class 'subprocess.CalledProcessError'>
OpenEWLD/dataset/Al_Dubin-Harry_Warren/September_in_the_Rain/September_in_the_Rain.mxl
C major 4/4
transposing from key C major to C major using interval 0


...

python: can't open file '/content/xml2abc_143/xml2abc.py': [Errno 2] No such file or directory
python: can't open file '/content/xml2abc_143/xml2abc.py': [Errno 2] No such file or directory


Unexpected error: <class 'subprocess.CalledProcessError'>
OpenEWLD/dataset/Will_Fyffe/I_Belong_to_Glasgow/I_Belong_to_Glasgow.mxl
D major 3/4
OpenEWLD/dataset/Bert_Kalmar-Harry_Ruby-Herbert_Stothart/I_Wanna_Be_Loved_by_You/I_Wanna_Be_Loved_by_You.mxl
C major 2/2
transposing from key C major to C major using interval 0
Unexpected error: <class 'subprocess.CalledProcessError'>
OpenEWLD/dataset/Ben_Pollack-George_Brunies-Leon_Roppolo-Mel_Stitzel-Paul_Mares-Walter_Melrose/Tin_Roof_Blues/Tin_Roof_Blues.mxl
E- major 4/4
transposing from key E- major to C major using interval -3


python: can't open file '/content/xml2abc_143/xml2abc.py': [Errno 2] No such file or directory
python: can't open file '/content/xml2abc_143/xml2abc.py': [Errno 2] No such file or directory


Unexpected error: <class 'subprocess.CalledProcessError'>
OpenEWLD/dataset/Andrew_B._Sterling-Harry_Von_Tilzer/Wait_Till_the_Sun_Shines_Nellie/Wait_Till_the_Sun_Shines_Nellie.mxl
C major 4/4
transposing from key C major to C major using interval 0
Unexpected error: <class 'subprocess.CalledProcessError'>
OpenEWLD/dataset/Lorenz_Hart-Richard_Rodgers/Here_in_My_Arms/Here_in_My_Arms.mxl
F major 4/4
transposing from key F major to C major using interval -5
Unexpected error: <class 'subprocess.CalledProcessError'>
OpenEWLD/dataset/Bert_Kalmar-Harry_Ruby-Ted_Snyder/Who's_Sorry_Now?/Who's_Sorry_Now?.mxl
B- major 2/2
transposing from key B- major to C major using interval 2
Unexpected error: <class 'subprocess.CalledProcessError'>
num prompts is 0


python: can't open file '/content/xml2abc_143/xml2abc.py': [Errno 2] No such file or directory
python: can't open file '/content/xml2abc_143/xml2abc.py': [Errno 2] No such file or directory


# **Check the Training File**

In [ ]:
!openai tools fine_tunes.prepare_data -f /content/songs.jsonl

# **Train GPT-3**

In [ ]:
!export OPENAI_API_KEY="<Your OpenAI API Key>"; openai api fine_tunes.create -t songs.jsonl --model curie --n_epochs 5

# **Generate Five Songs**

In [ ]:
import openai
import music21
import numpy as np
from collections.abc import Iterable
import numpy as np

CMM_mean = 2.2715
CMM_std = 0.4831

LM_mean = 2.0305
LM_std = 0.5386

CENT_mean = 0.3042
CENT_std = 0.0891

band_name = "I Lost My Voice"
song_name = "The Rare Pearls"
prompt = "X: 1 $ T: " + song_name + " $ C: " + band_name + " $ <song>"
print(prompt)
print()

openai.api_key = "<Your OpenAI API Key>"

songs_with_scores = []
score_arr = np.empty((0), np.float32)

for i in range(5):
    print("\n  Generating Song", i)
    response = openai.Completion.create(
      model="curie:ft-user-j0julqovorjakyuyt3kv3zci-2021-08-24-11-42-59",
      prompt=prompt,
      stop = " $ <end>",
      temperature=0.75,
      top_p=1.0,
      frequency_penalty=0.0,
      presence_penalty=0.0,
      max_tokens = 1000)

    print(response)
    print()

    formatted_prompt = "X: 1 $ T: " + song_name + " $ C: " + band_name + " $ L: 1/4 $ M: 4/4 $ K: C $ V: 1 treble"
    formatted_prompt = formatted_prompt.replace(" $ ", "\n")
    formatted_prompt = formatted_prompt.replace("<song>", "").strip()

    formatted_song = response["choices"][0]["text"].strip()
    formatted_song = formatted_song.replace('`', '"')
    formatted_song = formatted_song.replace(" $ ", "\n")
    new_song = formatted_prompt+ "\n" + formatted_song
    print(new_song)
    with open("new_song.abc", "w") as new_song_file:
    new_song_file.write(new_song)

    song = music21.converter.parse("new_song.abc")

    part = song.parts[0]
    chord_end = song.highestTime
    for pi in reversed(range(len(part))):
    p = part[pi]
    for ni in reversed(range(len(p))):
      n = p[ni]
      if type(n) == music21.harmony.ChordSymbol:
        chord_start = p.offset + n.offset
        n.duration.quarterLength = chord_end - chord_start
        n.volume = music21.volume.Volume(velocity=48)
        chord_end = chord_start
      elif type(n) == music21.note.Note:
        n.volume = music21.volume.Volume(velocity=64)
    file_name = "song" + str(i).zfill(2) + ".mid"
    song.write('midi', fp=file_name)

    part = song.parts[0]
    note_array = []

    for p in part:
    if isinstance(p, Iterable):
      for n in p:
        if type(n) == music21.note.Note:
          note_array.append([int(n.pitch.ps), int(n.quarterLength*4+0.5)])

    CMM = music_geometry_eval.calculate_time_supported_conjunct_melodic_motion(note_array)
    LM = music_geometry_eval.calculate_time_supported_limited_macroharmony(note_array, span_size=32)
    CENT = music_geometry_eval.calculate_time_supported_centricity(note_array, span_size=32)

    print("  CMM :", round(CMM, 4))
    print("  LM  :", round(LM, 4))
    print("  CENT:", round(CENT, 4))

    norm_cmm = (CMM - CMM_mean) / CMM_std
    norm_lm = (LM - LM_mean) / LM_std
    norm_cent = (CENT - CENT_mean) / CMM_std

    norm_score_squared = norm_cmm * norm_cmm + norm_lm * norm_lm + norm_cent * norm_cent
    print("  NDM:", round(norm_score_squared, 4))
    score_arr = np.append(score_arr, norm_score_squared)

    songs_with_scores.append([norm_score_squared, file_name])

songs_with_scores.sort()
for pair in songs_with_scores:
    print(round(pair[0], 4), pair[1])

# **Choose a Song and Play It**


In [ ]:
song_number = 0

melody_ns = note_seq.midi_file_to_sequence_proto(songs_with_scores[song_number][1])
print(round(songs_with_scores[song_number][0], 4), songs_with_scores[song_number][1])

note_seq.play_sequence( 
  melody_ns,
  synth=note_seq.fluidsynth, sample_rate=SAMPLE_RATE, sf2_path=SF2_PATH)
note_seq.plot_sequence(melody_ns)